In [1]:
pip install pandas networkx matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [ ]:
#load the csv files
